## MCP Client-Server: Simulación Local
> Vamos a simular un servidor MCP remoto, mientras nos conectamos desde nuestra app local, mediante Strands Agents

Source: [Strands Agents notebook](https://github.com/aws-samples/sample-bedrock-agentcore-with-strands-and-nova/blob/main/04-bedrock-agentcore-runtime-mcp/04-agentcore-runtime-for-mcp-deploy.ipynb)

In [ ]:
%%writefile mcp_server.py
from mcp.server.fastmcp import FastMCP
from ddgs import DDGS
from ddgs.exceptions import RatelimitException, DDGSException

mcp = FastMCP(host="0.0.0.0", stateless_http=True)

# Define a websearch tool
@mcp.tool()
def websearch(keywords: str, region: str = "us-en", max_results: int | None = None) -> list:
    """Search the web to get updated information.
    Args:
        keywords (str): The search query keywords.
        region (str): The search region: wt-wt, us-en, uk-en, ru-ru, etc..
        max_results (int | None): The maximum number of results to return.
    Returns:
        List of dictionaries with search results.
    """
    try:
        results = DDGS().text(keywords, region=region, max_results=max_results)
        return results if results else "No results found."
    except RatelimitException:
        return "RatelimitException: Please try again after a short delay."
    except DDGSException as d:
        return f"DuckDuckGoSearchException: {d}"
    except Exception as e:
        return f"Exception: {e}"

if __name__ == "__main__":
    mcp.run(transport="streamable-http")

> Start the server:
```
pip install -r requirements.txt
python mcp_server.py
```
or
```
uv pip install -r requirements.txt
uv run mcp_server.py
```

### Simulación de Cliente:

In [ ]:
import boto3

region = boto3.session.Session().region_name

NOVA_PRO_MODEL_ID = "us.amazon.nova-pro-v1:0"
if region.startswith("eu"):
    NOVA_PRO_MODEL_ID = "eu.amazon.nova-pro-v1:0"
elif region.startswith("ap"):
    NOVA_PRO_MODEL_ID = "apac.amazon.nova-pro-v1:0"

print(f"Nova Pro Model ID: {NOVA_PRO_MODEL_ID}")

In [ ]:
from strands import Agent
from strands.models import BedrockModel
from strands.tools.mcp import MCPClient
from mcp.client.streamable_http import streamablehttp_client

# Connect to the web search MCP server
print("\nConnecting to MCP Server...")
mcp_url = f"http://localhost:8000/mcp"
websearch_server = MCPClient(lambda: streamablehttp_client(mcp_url))

with websearch_server:
    mcp_tools = (websearch_server.list_tools_sync())
    print(f"Available MCP tools: {[tool.tool_name for tool in mcp_tools]}")

    # Create agent with self-built MCP tools
    agent = Agent(
        model=BedrockModel(model_id=NOVA_PRO_MODEL_ID),
        system_prompt = """You are a helpful assistant that provides concise responses.
                        If web search, always use websearch tool first.
                        """,
        tools=mcp_tools,
    )

    agent("¿Para qué sirve Amazon Bedrock AgentCore?")